<a href="https://colab.research.google.com/github/saheedniyi02/naijaweb/blob/main/push_to_hub.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install datasets tldextract

INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.6/97.6 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 12.8 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import gzip
import pandas as pd
import json
import torch
import tldextract
import huggingface_hub
from tqdm import tqdm
import datasets
from datasets import Dataset, load_dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification

### data

In [ ]:
file_path = "/content/drive/MyDrive/nairaland_webtext/processed_formatted/00000.jsonl.gz"

In [ ]:
list_data=[]
# Open the .json.gz file, decompress, and read the JSON content
with gzip.open(file_path, 'rt', encoding='utf-8') as f:
     for line in tqdm(f):
        data = json.loads(line)
        list_data.append(data)

270137it [00:32, 8323.30it/s] 


In [ ]:
data_df=pd.DataFrame(list_data)

In [ ]:
del list_data

In [ ]:
data_df["metadata"][0]#["url"]

{'url': 'https://www.vanguardngr.com/2021/03/ortom-commends-buhari-on-shoot-at-sight-order-on-ak47-bearing-criminals/amp/',
 'token_count': 332,
 'file_path': '/content/drive/MyDrive/nairaland_webtext/processed_final/00000.jsonl.gz'}

In [ ]:
data_df["link"]=[datapoint["url"] for datapoint in tqdm(data_df["metadata"])]

100%|██████████| 270137/270137 [00:00<00:00, 2130356.88it/s]


In [ ]:
data_df["token_count"]=[datapoint["token_count"] for datapoint in tqdm(data_df["metadata"])]

100%|██████████| 270137/270137 [00:00<00:00, 2231916.15it/s]


In [ ]:
data_df.drop("metadata", inplace=True,axis=1)

In [ ]:
data_df.columns

Index(['text', 'id', 'link', 'token_count'], dtype='object')

### get section

In [ ]:
links_df=pd.read_csv("/content/drive/MyDrive/nairaland_webtext/links/final_links.csv")

In [ ]:
links_df.head()

,link,section
0,http://www.baricblog.com/2013/01/start-trading...,Career
1,http://saharareporters.com/2021/05/20/break-ni...,Politics
2,http://accountingsoftware.com.ng/mortgage-back...,Business
3,http://www.punchng.com/news/pdp-primaries-jona...,Politics
4,https://www.vanguardngr.com/2020/12/i-saw-over...,Politics


In [ ]:
data_df.head()

,text,id,link,token_count
0,Governor Samuel Ortom of Benue State\nBy Peter...,/0,https://www.vanguardngr.com/2021/03/ortom-comm...,332
1,NewsHelm.com offers a unique blend of modernit...,1,http://www.newshelm.com/2016/08/photos-preside...,424
2,BY LEVINUS NWABUGHIOGU & EMMAN OVUAKPORIE\nABU...,3,http://www.vanguardngr.com/2013/11/icpc-recove...,875
3,Eyitayo Jegede and Jimoh Ibrahim\nBy Ikechukwu...,5,http://www.vanguardngr.com/2016/11/ondo-s-cour...,729
4,"Mrs. Helen Mark, wife of Senator David Mark, p...",6,http://www.vanguardngr.com/2016/04/mark-celebr...,545


In [ ]:
data_df=pd.merge(data_df,links_df,on="link",how="left")

In [ ]:
data_df["section"].value_counts()

,count
section,
Politics,131126
Sports,20739
Business,15478
Education,12084
Religion,11755
Health,11637
Romance,6131
Travel,5857
Phones,5230


In [ ]:
data_df[pd.isna(data_df["section"])]

,text,id,link,token_count,section
39177,*Buhari condemns killings in Kaduna\nBy Johnbo...,42488,https:////www.vanguardngr.com//2021//12//58-ki...,1271,NaN
39178,crowd of people\nBy Obadiah Mailafia\nBRITISH ...,42489,http:////www.vanguardngr.com//2017//07//smartes,1393,NaN
39179,Jonathan and Buhari\nBy Ben Agande\nAbuja — A ...,42490,http:////www.vanguardngr.com//2015//04//transi...,743,NaN
39180,Fed Govt draws up anti-oil theft battle plan\n...,42491,http:////thenationonlineng.net//fed-govt-draws...,1077,NaN
39181,The new price was set on Wednesday after a mee...,42492,http:////www.channelstv.com//2016//05//11//fg-...,601,NaN
...,...,...,...,...,...
270132,By Ibrahim Hassan-Wuyo\nAhead of the 2023 gene...,350089,https://www.vanguardngr.com/2022/07/muslim-lea...,824,NaN
270133,By Jacob Ajom\nA three-day programme aimed at ...,350090,https://www.vanguardngr.com/2023/04/global-,325,NaN
270134,"March 18, 2021 by The Nation\nBiafra: Uwazurui...",350092,https://thenationonlineng.net/biafra-uwazuruik...,491,NaN
270135,"1st May, 2022\nNobel laureate, Prof. Wole Soyi...",350093,https://pmnewsnigeria.com/2022/05/01/breaking-...,327,NaN


In [ ]:
def get_domain(url):
    url=clean_url(url)
    ext = tldextract.extract(url)
    #domain = ext.domain+'.'+ext.suffix
    return ext.registered_domain

def clean_url(url):
    url=url.replace("http:////","http://")
    url=url.replace("https:////","https://")
    url=url.replace("http:///","http://")
    url=url.replace("https:///","https://")

    return url

In [ ]:
data_df["domain"]=data_df["link"].apply(get_domain)

In [ ]:
data_df["domain"].value_counts().head(50)

,count
domain,
vanguardngr.com,29528
punchng.com,20463
dailypost.ng,11193
thenationonlineng.net,8992
thecable.ng,5481
brandspurng.com,5206
dailytrust.com,4342
channelstv.com,3969
dailytrust.com.ng,3860


In [ ]:
data_df.to_csv("data.csv")

In [ ]:
dataset=load_dataset("csv",data_files="data.csv")

Generating train split: 0 examples [00:00, ? examples/s]

In [ ]:
dataset

DatasetDict({
    train: Dataset({
        features: ['Unnamed: 0', 'text', 'id', 'link', 'token_count', 'section', 'domain'],
        num_rows: 270137
    })
})

In [ ]:
model_name="HuggingFaceFW/fineweb-edu-classifier"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(
        model_name, torch_dtype=torch.bfloat16
    )

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.38k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/712k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/695 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/804 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.0, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

In [ ]:
def compute_scores(batch):
        inputs = tokenizer(
            batch["text"],
            return_tensors="pt",
            padding="longest",
            truncation=True,
        ).to(device)
        with torch.no_grad():
            outputs = model(**inputs)
            logits = outputs.logits.squeeze(-1).float().cpu().numpy()

        batch["score"] = logits.tolist()
        batch["int_score"] = [int(round(max(0, min(score, 5)))) for score in logits]
        return batch

In [ ]:
dataset = dataset.map(compute_scores, batched=True, batch_size=512)

Map:   0%|          | 0/270137 [00:00<?, ? examples/s]

In [ ]:
dataset=dataset.remove_columns(['Unnamed: 0', 'id','domain','score'])

In [ ]:
huggingface_hub.login()

In [ ]:
dataset.push_to_hub("fineweb",private =True)

Uploading the dataset shards:   0%|          | 0/3 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/91 [00:00<?, ?ba/s]

Creating parquet from Arrow format:   0%|          | 0/91 [00:00<?, ?ba/s]

Creating parquet from Arrow format:   0%|          | 0/91 [00:00<?, ?ba/s]

README.md:   0%|          | 0.00/530 [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/datasets/saheedniyi/testweb/commit/1cae5b088bee20b64a318e49a5e4c11a8567377c', commit_message='Upload dataset', commit_description='', oid='1cae5b088bee20b64a318e49a5e4c11a8567377c', pr_url=None, pr_revision=None, pr_num=None)